# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [9]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [10]:
# Read the Mapbox API key
load_dotenv()
mapbox_token = os.getenv("MAPBOX_API_KEY")

# Import Data

In [11]:
# Import the CSVs to Pandas DataFrames
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/neighborhoods_coordinates.csv")
df_neighborhood_locations = pd.read_csv(file_path)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [59]:
# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""
    
    # YOUR CODE HERE!
    sfo_housing = sfo_data["housing_units"]
    fig_housing_units = sfo_housing.groupby(sfo_housing.index).mean()
    
    min_year = min(fig_housing_units.index)
    max_year = max(fig_housing_units.index)
    std_sfo = fig_housing_units.std()
    min_y = min(fig_housing_units) - std_sfo
    max_y = max(fig_housing_units) + std_sfo


    plt = fig_housing_units.hvplot.bar(ylim=(min_y, max_y),\
        title = f"Housing Units in San Francisco from {min_year} to {max_year}").opts(yformatter = "%.0f")
    return plt

def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    
    # YOUR CODE HERE!
    sfo_rent_sale = sfo_data[["gross_rent","sale_price_sqr_foot"]]
    sfo_rs_grouped = (sfo_rent_sale.groupby(sfo_rent_sale.index)
                 .mean()
                 .reset_index()
    )
    plt_gr = sfo_rs_grouped["gross_rent"].hvplot(title = "Average Gross Rent in San Francisco")
    return plt_gr

def average_sales_price():
    """Average Sales Price Per Year."""
    
    # YOUR CODE HERE!
    sfo_rent_sale = sfo_data[["gross_rent","sale_price_sqr_foot"]]
    sfo_rs_grouped = (sfo_rent_sale.groupby(sfo_rent_sale.index)
                 .mean()
                 .reset_index()
    )
    plt_asp = sfo_rs_grouped["sale_price_sqr_foot"].hvplot(title = "Average Sales Price Per Year in San Francisco")
    return plt_asp

def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    
    # YOUR CODE HERE!
    sfo_nbh = sfo_data.groupby([sfo_data.index,"neighborhood"]).mean()
    plt_nbh = sfo_nbh["sale_price_sqr_foot"]
    plt = plt_nbh.hvplot(title = "Sale Price Per Sq Ft", groupby = "neighborhood")
    return plt

def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""
    
    # YOUR CODE HERE!
    sfo_top10 = sfo_data[["neighborhood","sale_price_sqr_foot"]]\
                          .reset_index()

    sfo_top10 = sfo_top10.drop('year',1)
    sfo_top10_grp = (sfo_top10.groupby("neighborhood").mean()
                                     .sort_values("sale_price_sqr_foot",ascending = False).head(10)
    )
    plt_top10 = (sfo_top10_grp.hvplot.bar(height = 400, title = "Top 10 Expensive Neighborhoods in SFO", ylabel = "Avg Sale Price Per Sq Ft",rot = 90, ylim = (200,1000)))
    return plt_top10

def parallel_coordinates():
    """Parallel Coordinates Plot."""
    
    # YOUR CODE HERE!
    sfo_nbh = sfo_data.groupby([sfo_data.index,"neighborhood"]).mean()
    plt_coord = sfo_nbh.groupby("neighborhood").mean().sort_values("sale_price_sqr_foot",ascending = False).reset_index().head(10)
    plt = px.parallel_coordinates(plt_coord,
                                  color = "sale_price_sqr_foot",
                                  title = "Parallel Coordinates Plot",
                                  labels = {
                                    "sale_price_sqr_foot": "Price/Sq Ft",
                                    "housing_units": "Housing Units",
                                    "gross_rent" : "Gross Rent"
                                  }
    )
    return plt

def parallel_categories():
    """Parallel Categories Plot."""
    
    # YOUR CODE HERE!
    sfo_nbh = sfo_data.groupby([sfo_data.index,"neighborhood"]).mean()
    plt_coord = sfo_nbh.groupby("neighborhood").mean().sort_values("sale_price_sqr_foot",ascending = False).reset_index().head(10)
    
    plt_coord["sale_price_sqr_foot"] = round(plt_coord["sale_price_sqr_foot"],2)
    plt_coord["gross_rent"] = round(plt_coord["gross_rent"],2)
    
    plt = px.parallel_categories(
        plt_coord,
        dimensions=["neighborhood","sale_price_sqr_foot", "gross_rent", "housing_units"],
        color="sale_price_sqr_foot",
        color_continuous_scale=px.colors.sequential.Inferno,
        labels={
            "neighborhood": "Neighborhood",
            "sale_price_sqr_foot": "Price/Sq Ft",
            "gross_rent": "Gross Rent",
        },
        title = "Parallel Categories Plot",
    )
    return plt 

def neighborhood_map():
    """Neighborhood Map"""
    px.set_mapbox_access_token(mapbox_token)
    # YOUR CODE HERE!
    df_nbh_mean = (sfo_data.groupby("neighborhood")
              .mean()
              .reset_index()
    )

    df_concat = pd.concat([df_neighborhood_locations,df_nbh_mean],axis = 1)
    df_concat = df_concat[["Neighborhood","Lat","Lon","sale_price_sqr_foot","housing_units","gross_rent"]]
    
    map_nbh = px.scatter_mapbox(
        df_concat,
        lat= "Lat",
        lon="Lon",
        size="gross_rent",
        color="sale_price_sqr_foot",
        width = 900,
        zoom= 10.5
    )
    return map_nbh

#housing_units_per_year
#average_gross_rent()
#average_sales_price()
#average_price_by_neighborhood()
#top_most_expensive_neighborhoods()
#parallel_coordinates()
#parallel_categories()
#neighborhood_map()

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [61]:
# YOUR CODE HERE!

pc_row = pn.Row(parallel_coordinates(),parallel_categories())
hu_col = pn.Column("<h2>Total Housing Units in SFO<h2>",housing_units_per_year())
cost_row1 = pn.Row(average_gross_rent(),average_sales_price())
cost_row2 = pn.Row(top_most_expensive_neighborhoods(), average_price_by_neighborhood())
cost_col1 = pn.Column("<h2>Cost Plots for SFO</h2>",cost_row1,cost_row2)
map_col = pn.Column("<h2>Heat Map of SFO Neighborhoods<h2>",neighborhood_map())


# Create tabs
panel = pn.Tabs(
    ("Volume", hu_col),
    ("Cost", cost_col1),
    ("Parallel", pc_row),
    ("Map", map_col)
)
panel

Tabs
    [0] Column
        [0] Markdown(str)
        [1] HoloViews(Bars)
    [1] Column
        [0] Markdown(str)
        [1] Row
            [0] HoloViews(Curve)
            [1] HoloViews(Curve)
        [2] Row
            [0] HoloViews(Bars)
            [1] Row
                [0] HoloViews(DynamicMap)
                [1] Column
                    [0] WidgetBox
                        [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                    [1] VSpacer()
    [2] Row
        [0] Plotly(Figure)
        [1] Plotly(Figure)
    [3] Column
        [0] Markdown(str)
        [1] Plotly(Figure)

## Serve the Panel Dashboard

In [62]:
panel.servable()

Tabs
    [0] Column
        [0] Markdown(str)
        [1] HoloViews(Bars)
    [1] Column
        [0] Markdown(str)
        [1] Row
            [0] HoloViews(Curve)
            [1] HoloViews(Curve)
        [2] Row
            [0] HoloViews(Bars)
            [1] Row
                [0] HoloViews(DynamicMap)
                [1] Column
                    [0] WidgetBox
                        [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                    [1] VSpacer()
    [2] Row
        [0] Plotly(Figure)
        [1] Plotly(Figure)
    [3] Column
        [0] Markdown(str)
        [1] Plotly(Figure)